In [1]:
from unsloth import FastModel
import torch
from torch.nn.utils import clip_grad_value_
import torch.nn as nn
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=16,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-01 20:36:32 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
model = model.base_model
model.train();
model.lm_head.weight.requires_grad_(True);

In [3]:
# resize model vocab
# model.model.embed_tokens = Gemma3TextScaledWordEmbedding(16,2560,0,50.59644256269407)
# model.lm_head = nn.Linear(2560, 16, bias=False)
# model.lm_head.weight = model.model.embed_tokens.weight

In [1]:
import json
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'r') as f:
    data = json.load(f)

#### Fine-tune embedding

In [ ]:
epochs = 10
accumulation_steps = 32
lr = 1e-5
clip = 1e-3
MAX_LEN = 3600

In [6]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# 85 minutes
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1 - Train Loss: 7.0098 - Val Loss: 8.8345
Epoch 2 - Train Loss: 6.8387 - Val Loss: 8.6338
Epoch 3 - Train Loss: 6.8292 - Val Loss: 8.4646
Epoch 4 - Train Loss: 6.7137 - Val Loss: 8.3336
Epoch 5 - Train Loss: 6.8827 - Val Loss: 8.2291
Epoch 6 - Train Loss: 6.5764 - Val Loss: 8.1309
Epoch 7 - Train Loss: 6.5341 - Val Loss: 8.0598
Epoch 8 - Train Loss: 6.4691 - Val Loss: 7.9518
Epoch 9 - Train Loss: 6.3932 - Val Loss: 7.8759
Epoch 10 - Train Loss: 6.4143 - Val Loss: 7.8158


In [8]:
new_lr = lr * 3
clip *= 3
for param_group in optimizer.param_groups:
    param_group['lr'] = new_lr

In [9]:
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

Epoch 1 - Train Loss: 5.7738 - Val Loss: 6.1679
Epoch 2 - Train Loss: 4.7226 - Val Loss: 4.5743
Epoch 3 - Train Loss: 4.0356 - Val Loss: 4.1086
Epoch 4 - Train Loss: 3.7343 - Val Loss: 3.8418
Epoch 5 - Train Loss: 3.5611 - Val Loss: 3.6761
Epoch 6 - Train Loss: 3.3778 - Val Loss: 3.5208
Epoch 7 - Train Loss: 3.2983 - Val Loss: 3.3679
Epoch 8 - Train Loss: 3.1608 - Val Loss: 3.2395
Epoch 9 - Train Loss: 3.0113 - Val Loss: 3.1285
Epoch 10 - Train Loss: 2.8824 - Val Loss: 3.0360


In [10]:
torch.save(model.lm_head.state_dict(), '../Model/lm_heads_weights.pth')

#### Fine-Tune QLORA

In [ ]:
epochs = 10
accumulation_steps = 32
lr = 2e-5
clip = 2e-3
MAX_LEN = 3600

In [5]:
model.lm_head.load_state_dict(torch.load('../Model/lm_heads_weights.pth'))

<All keys matched successfully>

In [ ]:
model.max_seq_length = MAX_LEN

In [5]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           # Larger = higher accuracy, but might overfit
    lora_alpha = 64,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [6]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [9]:
train_loss = 0
val_loss = 0
for epoch in range(epochs):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1 - Train Loss: 1.9318 - Val Loss: 1.5444
Epoch 2 - Train Loss: 1.3650 - Val Loss: 1.1297
Epoch 3 - Train Loss: 1.1207 - Val Loss: 0.8844
Epoch 4 - Train Loss: 0.9271 - Val Loss: 0.7157
Epoch 5 - Train Loss: 0.7849 - Val Loss: 0.6018
Epoch 6 - Train Loss: 0.6744 - Val Loss: 0.5305
Epoch 7 - Train Loss: 0.6055 - Val Loss: 0.4783
Epoch 8 - Train Loss: 0.5448 - Val Loss: 0.4438
Epoch 9 - Train Loss: 0.4968 - Val Loss: 0.4217
Epoch 10 - Train Loss: 0.4682 - Val Loss: 0.4048


In [10]:
train_loss = 0
val_loss = 0
for epoch in range(5):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

Epoch 1 - Train Loss: 0.4530 - Val Loss: 0.3874
Epoch 2 - Train Loss: 0.4321 - Val Loss: 0.3754
Epoch 3 - Train Loss: 0.4121 - Val Loss: 0.3622
Epoch 4 - Train Loss: 0.4018 - Val Loss: 0.3536
Epoch 5 - Train Loss: 0.3931 - Val Loss: 0.3469


In [11]:
train_loss = 0
val_loss = 0
for epoch in range(5):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

Epoch 1 - Train Loss: 0.3856 - Val Loss: 0.3393
Epoch 2 - Train Loss: 0.3728 - Val Loss: 0.3375
Epoch 3 - Train Loss: 0.3648 - Val Loss: 0.3270
Epoch 4 - Train Loss: 0.3581 - Val Loss: 0.3217
Epoch 5 - Train Loss: 0.3484 - Val Loss: 0.3178


In [12]:
train_loss = 0
val_loss = 0
for epoch in range(5):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

Epoch 1 - Train Loss: 0.3385 - Val Loss: 0.3157
Epoch 2 - Train Loss: 0.3326 - Val Loss: 0.3118
Epoch 3 - Train Loss: 0.3300 - Val Loss: 0.3061
Epoch 4 - Train Loss: 0.3215 - Val Loss: 0.3039
Epoch 5 - Train Loss: 0.3179 - Val Loss: 0.3025


In [13]:
train_loss = 0
val_loss = 0
for epoch in range(5):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

Epoch 1 - Train Loss: 0.3149 - Val Loss: 0.2965
Epoch 2 - Train Loss: 0.3101 - Val Loss: 0.2938
Epoch 3 - Train Loss: 0.3050 - Val Loss: 0.2910
Epoch 4 - Train Loss: 0.3007 - Val Loss: 0.2938
Epoch 5 - Train Loss: 0.2999 - Val Loss: 0.2891


In [ ]:
model.save_pretrained("../Model/merged_model")
torch.save(model.lm_head.state_dict(), '../Model/lm_heads_weights.pth')

/home/zhenlan/anaconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in  - will assume that the vocabulary was not modified.
  warnings.warn(


#### Load model

In [2]:
from unsloth import FastModel,FastLanguageModel
import torch
from torch.nn.utils import clip_grad_value_
import torch.nn as nn
import gc
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=16,
)
model = model.base_model
model.lm_head.weight.requires_grad_(True);
gc.collect()
torch.cuda.empty_cache()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-02 06:15:03 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
model.lm_head.load_state_dict(torch.load('../Model/lm_heads_weights.pth'))

<All keys matched successfully>

In [4]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "../Model/merged_model/",is_trainable=True)

In [5]:
epochs = 10
accumulation_steps = 32
lr = 2e-5
clip = 2e-3
MAX_LEN = 3600

In [6]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [7]:
model.max_seq_length = MAX_LEN

In [8]:
train_loss = 0
val_loss = 0
for epoch in range(5):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1 - Train Loss: 0.2936 - Val Loss: 0.2870
Epoch 2 - Train Loss: 0.2905 - Val Loss: 0.2853
Epoch 3 - Train Loss: 0.2879 - Val Loss: 0.2837
Epoch 4 - Train Loss: 0.2859 - Val Loss: 0.2828
Epoch 5 - Train Loss: 0.2793 - Val Loss: 0.2755


In [9]:
train_loss = 0
val_loss = 0
for epoch in range(5):
    model.train()
    for i,(x,y) in enumerate(data_gen(data,True,MAX_LEN)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(x).logits
            loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        loss.backward()
        train_loss += loss.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(x,y) in enumerate(data_gen(data,False,MAX_LEN*4)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(x).logits
                loss = loss_fn(yhat.view(-1,yhat.shape[-1]),y.view(-1))
        val_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Train Loss: {train_loss/i:.4f} - Val Loss: {val_loss/j:.4f}")
    train_loss = 0
    val_loss = 0

Epoch 1 - Train Loss: 0.2782 - Val Loss: 0.2764
Epoch 2 - Train Loss: 0.2719 - Val Loss: 0.2764
Epoch 3 - Train Loss: 0.2742 - Val Loss: 0.2792
Epoch 4 - Train Loss: 0.2713 - Val Loss: 0.2724
Epoch 5 - Train Loss: 0.2657 - Val Loss: 0.2711


In [10]:
model.save_pretrained("../Model/merged_model")
torch.save(model.lm_head.state_dict(), '../Model/lm_heads_weights.pth')

/home/zhenlan/anaconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in  - will assume that the vocabulary was not modified.
  warnings.warn(
